# Classification Problem
This notebook will try to solve direction classification problem.
The Data phone sensors acceleration & gyroscope
The Target we will try to find the direction the phone moved - up, down, left, right, up left, up right, down left, down right 

In [3]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Dropout, LeakyReLU, Reshape, Bidirectional, LSTM, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import concatenate, Conv1D, BatchNormalization, Flatten, Multiply, Concatenate, Add, Subtract, Dot, Lambda, PReLU, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf

In [4]:
BATCH_SIZE = 32
EPOCKS = 50

# Loading the data

In [5]:
def load_data():
    path = '../input/generations'
    gens = None
    paths = os.listdir(path)
    for p in paths:
        gen = pd.read_csv(path + '/' +p)
        if gens is None:
            gens = gen
        else:
            gens = pd.concat([gens, gen])
    return gens

gens = load_data()
gens.head()

,Unnamed: 0,tag,accelometer_x,accelometer_y,accelometer_z,gyroscope_x,gyroscope_y,gyroscope_z,speed
0,0,up,-0.457306,3.747040,9.706390,0.001173,-0.012273,0.005558,NaN
1,1,up,-0.629694,3.845205,9.775824,-0.000659,-0.004943,0.003725,NaN
2,2,up,0.881093,2.636096,9.806950,-0.006768,0.013994,0.072753,NaN
3,3,up,-0.766168,2.188367,9.696813,-0.004324,-0.001888,0.292053,NaN
4,4,up,-0.131685,1.106155,9.885961,-0.002492,-0.002499,0.283501,NaN


# Analyzing The data & Split to train and test sets

In [6]:
def analyze_tags(gens):
    tags = gens['tag'].unique()
    print('tags (', len(tags) ,'): ', tags)
    grouped = gens.groupby('tag').count()

analyze_tags(gens)

tags ( 8 ):  ['up' 'down' 'right' 'left' 'upright' 'downleft' 'upleft' 'downright']


In [7]:
def create_X_y(gens):
    X = gens.drop(['tag','speed','Unnamed: 0'], axis=1)
    y = gens['tag']
    return X, y

X, y = create_X_y(gens)

In [8]:
def split_train_test(X, y, precentage = 0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=precentage)
    print('X train shape: ',X_train.shape, ' y train shape: ', y_train.shape)
    print('X test shape: ',X_test.shape, ' y test shape: ', y_test.shape)
    return X_train, X_test, y_train, y_test

def y_to_categorical(y_train, y_test):
    dic = {x:i for i,x in enumerate(y_train.unique())}
    len_train = len(y_train)
    y = np.concatenate([y_train.to_numpy(), y_test.to_numpy()])
    y = [dic[x] for x in y]
    y = to_categorical(y)
    y_train = y[0:len_train]
    y_test = [dic[x] for x in y_test]
    print('Changed to catigorical','y train shape: ',len(y_train), ' y test shape: ', len(y_test))
    return y_train, y_test, dic

X_train, X_test, y_train, y_test = split_train_test(X, y)
y_train, y_test, y_dic = y_to_categorical(y_train, y_test)

X train shape:  (4739, 6)  y train shape:  (4739,)
X test shape:  (1185, 6)  y test shape:  (1185,)
Changed to catigorical y train shape:  4739  y test shape:  1185


In [9]:
def split_train_test_by_tag(gens, precentage = 0.2):
    grouped = gens.groupby('tag')
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    for g in grouped:
        X, y = create_X_y(g[1])
        X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(X, y)
        if X_train is None:
            X_train, X_test, y_train, y_test = X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp 
        else:
            X_train = pd.concat([X_train, X_train_tmp])
            y_train = pd.concat([y_train, y_train_tmp])
            X_test = pd.concat([X_test, X_test_tmp])
            y_test = pd.concat([y_test, y_test_tmp])
    print('X grouped train shape: ',X_train.shape, ' y grouped train shape: ', y_train.shape)
    print('X grouped test shape: ',X_test.shape, ' y grouped test shape: ', y_test.shape)
    return X_train, X_test, y_train, y_test


X_train_g, X_test_g, y_train_g, y_test_g = split_train_test_by_tag(gens)
y_train_g, y_test_g, y_dic_g = y_to_categorical(y_train_g, y_test_g)

X grouped train shape:  (4441, 6)  y grouped train shape:  (4441,)
X grouped test shape:  (1483, 6)  y grouped test shape:  (1483,)
Changed to catigorical y train shape:  4441  y test shape:  1483


In [10]:
def split_train_test_by_subtruction_close_points(X,y):
    grouped = gens.groupby('tag')
    X_tmp = None
    y_tmp = None
    X_original = None
    for g in grouped:
        X, y = create_X_y(g[1])
        tmp_dic = {}
        if X_original is None:
            X_original = X.iloc[1:]
        else:
            X_original = pd.concat([X_original,X.iloc[1:]])
        for col in X.columns:
            tmp_dic['sub_'+col] = X[col].iloc[1:].values - X[col].iloc[:len(X)-1].values
        if X_tmp is None:
            X_tmp = pd.DataFrame.from_dict(tmp_dic)
            y_tmp = y.iloc[:len(X)-1]
        else:
            X_tmp = pd.concat([X_tmp, pd.DataFrame.from_dict(tmp_dic)])
            y_tmp = pd.concat([y_tmp, y.iloc[1:]])
    X_original = X_original.reset_index()
    X_tmp = X_tmp.reset_index()
    X_tmp = pd.concat([X_tmp, X_original], axis=1)
    X_tmp = X_tmp.drop(['index'], axis=1)
    print('X subtract: ',X_tmp.shape, ' y substract: ', y_tmp.shape)
    X_train, X_test, y_train, y_test = split_train_test(X_tmp, y_tmp)
    return X_train, X_test, y_train, y_test

X_train_s, X_test_s, y_train_s, y_test_s = split_train_test_by_subtruction_close_points(X, y)
y_train_s, y_test_s, y_dic_s = y_to_categorical(y_train_s, y_test_s)

X subtract:  (5916, 12)  y substract:  (5916,)
X train shape:  (4732, 12)  y train shape:  (4732,)
X test shape:  (1184, 12)  y test shape:  (1184,)
Changed to catigorical y train shape:  4732  y test shape:  1184


In [56]:
def split_train_test_by_sliding_window(X,y, N=2):
    grouped = gens.groupby('tag')
    X_tmp = None
    X_original = None
    y_original = None
    start = None
    end = None
    for g in grouped:
        X, y = create_X_y(g[1])
        start = N
        end = len(X)
        tmp_dic = {}
        if X_original is None:
            X_original = X.iloc[start-1:]
            y_original = y.iloc[start-1:]
        else:
            X_original = pd.concat([X_original,X.iloc[start-1:]])
            y_original = pd.concat([y_original, y.iloc[start-1:]])
        for col in X.columns:
            for i in range(N-1,0,-1):
                name = 'prev_'+str(i)+'_'+col
                tmp_dic[name] = X[col].iloc[start-i: end-i].values
        if X_tmp is None:
            X_tmp = pd.DataFrame.from_dict(tmp_dic)
        else:
            X_tmp = pd.concat([X_tmp, pd.DataFrame.from_dict(tmp_dic)])
    X_original = X_original.reset_index()
    X_tmp = X_tmp.reset_index()
    X_tmp = pd.concat([X_tmp, X_original], axis=1)
    X_tmp = X_tmp.drop(['index'], axis=1)
    print('X subtract: ',X_tmp.shape, ' y substract: ', y_original.shape)
    X_train, X_test, y_train, y_test = split_train_test(X_tmp, y_original)
    return X_train, X_test, y_train, y_test

X_train_d, X_test_d, y_train_d, y_test_d = split_train_test_by_sliding_window(X, y)
y_train_d, y_test_d, y_dic_d = y_to_categorical(y_train_d, y_test_d)

X subtract:  (5916, 12)  y substract:  (5916,)
X train shape:  (4732, 12)  y train shape:  (4732,)
X test shape:  (1184, 12)  y test shape:  (1184,)
Changed to catigorical y train shape:  4732  y test shape:  1184


# Creating Classification Naive Model 

In [ ]:
def create_callbacks(name, patience=5):
    early_stopping = EarlyStopping(patience=patience)
    cheak_point = ModelCheckpoint(name)
    return [early_stopping, cheak_point]

def create_metrics():
    return ['accuracy']

def create_model1(size=6):
    inp = Input(shape=(None,size))
    x = Dense(32, activation='relu')(inp)
    x = Dense(64, activation='relu')(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model1') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model1 = create_model1()
callbacks = create_callbacks('model1.h5')
history = model1.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.15, callbacks=callbacks)

## X,y not grouped 

In [ ]:
def quick_plot_loss(history, field, metric, ax):
    # Plot training & validation loss values
    ax.plot(history.history[field])
    ax.plot(history.history['val_'+field])
    ax.set_title('Model '+ metric)
    ax.set_ylabel(metric)
    ax.set_xlabel('Epoch')
    ax.legend(['Train', 'Validation'], loc='upper left')
                
    
def quick_plot_history(history):
    fig = plt.figure(figsize=(18, 4))
    ax = fig.add_subplot(1, 2, 1)
    ax.set_title('loss')
    quick_plot_loss(history, 'loss', 'categorical_crossentropy', ax)
    ax = fig.add_subplot(1, 2, 2)
    ax.set_title('accrucy')
    quick_plot_loss(history, 'accuracy', 'accuracy', ax)
    
quick_plot_history(history)

In [ ]:
def test_results(model,X_test,y_test, LOAD=True):
    preds = model.predict(X_test)
    pred_cat = np.argmax(preds,axis=1) #takes the maximum prediction and compare it to the real prediction
    acc = accuracy_score(y_test,pred_cat)*100
    acc_saved = -1
    if LOAD:
        saved_model = load_model('./'+model.name+'.h5')
        preds = saved_model.predict(X_test)
        pred_cat = np.argmax(preds,axis=1) #takes the maximum prediction and compare it to the real prediction
        acc_saved = accuracy_score(y_test,pred_cat)*100
    
    if acc > acc_saved:
        if LOAD:
            model.save('./'+model.name+'.h5')
        print('NEW: model accuracy on test set is: {0:.2f}%'.format(acc))
    else:
        print('model accuracy on test set is: {0:.2f}%'.format(acc_saved))

test_results(model1, X_test, y_test)

## X,y grouped 

In [ ]:
model1_grouped = create_model1()
callbacks = create_callbacks('model1_grouped.h5')
history = model1_grouped.fit(X_train_g, y_train_g, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.15)

In [ ]:
quick_plot_history(history)

It's seems for the naive model The random split is much better then the group split, The group split made the model to be overfitting.

# Improvments - Building More complex models
* model2 - added Droupout
* model3 - added more nyrones to model 2
* model4 - change the model architecture, added another dense moved the dropout to be the last, changed the first activation to tanh
* model5 - change the model architecture to support LSTM, added Reshape
* model6 - addeed more complexity model5

# Model 2

In [ ]:
def dense_block(inp, n=32):
    x = Dense(n, activation='relu')(inp)
    x = Dense(n*2, activation='relu')(x)
    return x

def create_model2(size=6):
    inp = Input(shape=(None,size))
    x = dense_block(inp)
    x = Dropout(0.2)(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model2') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model2 = create_model2()
callbacks = create_callbacks('model2.h5')
history = model2.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.15, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model2, X_test, y_test)

It seems only adding a dropout improved the results on the test set from the naive model.

# Model 3

In [ ]:
def create_model3(size=6):
    inp = Input(shape=(None,size))
    x = dense_block(inp, n=64)
    x = Dropout(0.2)(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model3') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model3 = create_model3()
callbacks = create_callbacks('model3.h5')
history = model3.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.15, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model3, X_test, y_test)

# Model 4

In [ ]:
def create_model4(size=6):
    inp = Input(shape=(None,size))
    x = Dense(16, activation='tanh')(inp)
    x = Dense(32, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model4') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model4 = create_model4()
callbacks = create_callbacks('model4.h5')
history = model4.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model4, X_test, y_test)

# Model 5

In [ ]:
def create_model5(size=6):
    inp = Input(shape=(None,size))
    x = Dense(256, activation='tanh')(inp)
    x = Reshape([4,64])(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Bidirectional(LSTM(16))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model5') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model5 = create_model5()
callbacks = create_callbacks('model5.h5')
history = model5.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model5, X_test, y_test)

# Model 6

In [ ]:
def create_model6(size=6):
    inp = Input(shape=(None,size))
    x = Dense(1024, activation='tanh')(inp)
    x = Reshape([16,64])(x)
    x = BatchNormalization(momentum=0.5)(x)
    x1 = Bidirectional(LSTM(64))(x)
    x1 = Flatten()(x1)
    x2 = GlobalAveragePooling1D()(x)
    x3 = GlobalMaxPooling1D()(x)
    x = concatenate([x1,x2,x3])
    x = Dense(256, activation='relu')(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model6') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model6 = create_model6()
callbacks = create_callbacks('model6.h5')
history = model6.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model6, X_test, y_test)

# Model 7

In [ ]:
def cov_blocks(inp, n, k):
    x = Conv1D(n, kernel_size=k, padding='same', activation='relu')(inp)
    x = Conv1D(n*2, kernel_size=k, padding='same', activation='relu')(x)
    x = Conv1D(n*3, kernel_size=k, padding='same', activation='relu')(x)
    x = Conv1D(n*4, kernel_size=k, padding='same', activation='relu')(x)
    x = Conv1D(n*5, kernel_size=k, padding='same', activation='relu')(x)
    x = Flatten()(x)
    return x

def create_model7(size=6):
    inp = Input(shape=(None,size))
    x = Dense(256, activation='tanh')(inp)
    x = Reshape([16,16])(x)
    x1 = cov_blocks(x, 16, 3)
    x2 = cov_blocks(x, 16, 4)
    x3 = cov_blocks(x, 16, 5)
    x = concatenate([x1,x2,x3])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x, name='model7') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model7 = create_model7()
callbacks = create_callbacks('model7.h5')
history = model7.fit(X_train, y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
test_results(model7, X_test, y_test)

## we will try creating a semaic model

# Model 8

In [ ]:
# Multiply, Concatenate, Add, Subtract
def seimaic(size=3):
    inp = Input(shape=size)
    x = Dense(16, activation='relu')(inp)
    x = Dense(32, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    return inp, x

def create_model_seamic1():
    inp1, s1 = seimaic()
    inp2, s2 = seimaic()
    x = concatenate([s1, s2])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2], outputs=x, name='model8') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model8 = create_model_seamic1()

In [ ]:
def split_semaic(X):
    A = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z'], axis=1)
    B = X.drop(['accelometer_x','accelometer_y','accelometer_z'], axis=1)
    return A, B

A, B = split_semaic(X_train)
callbacks = create_callbacks('model8.h5')
history = model8.fit([A,B], y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
A, B = split_semaic(X_test)
test_results(model8, [A,B], y_test)

# Model 9

In [ ]:
def dense_deapth_block(inp, N=32, extend_N=4, deapth=5):
    if deapth==0:
        return inp
    x = None 
    prev_x = None
    xs = []
    for i in range(0, deapth):
        if x is None:
            x = Dense(N)(inp)
            x = PReLU()(x)
        else:
            prev_x = x
            x = Dense(N + i*extend_N)(prev_x)
            x = PReLU()(x)
        xs.append(x)
    if len(xs) != 1:
        x = concatenate(xs)
    return x

def seimaic2(size=3, deapth=5):
    inp = Input(shape=size)
    x = dense_deapth_block(inp, deapth=deapth)
    return inp, x

def create_model_seamic2():
    inp1, s1 = seimaic2(deapth=2)
    inp2, s2 = seimaic2()
    x = concatenate([s1, s2])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2], outputs=x, name='model9') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model9 = create_model_seamic2()

In [ ]:
A, B = split_semaic(X_train)
callbacks = create_callbacks('model9.h5')
history = model9.fit([A,B], y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)

In [ ]:
A, B = split_semaic(X_test)
test_results(model9, [A,B], y_test)

# Creating a Bench Mark

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators = 100, max_depth = 8)
forest.fit(X_train,y_train)
test_results(forest, X_test, y_test, LOAD=False)

# Now that we pass the bench mark we will try to improve the model by extend the input model from 2 diffrent input chanels to 3,4,5 channels

# Model 10

In [ ]:
def split_channels_3(X):
    A = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z'], axis=1)
    B = X.drop(['accelometer_x','accelometer_y','accelometer_z'], axis=1)
    C = X.copy()
    return A, B, C

def create_chanel(size=3, deapth=5, N=32,extend_N=4):
    inp = Input(shape=size)
    x = dense_deapth_block(inp, deapth=deapth, N=N, extend_N=extend_N)
    return inp, x

def create_model_channel_3():
    inp1, c1 = create_chanel(deapth=2,N=64)
    inp2, c2 = create_chanel(N=64)
    inp3, c3 = create_chanel(size=6,deapth=2, N=12)
    x = concatenate([c1, c2, c3])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2, inp3], outputs=x, name='model10') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model10 = create_model_channel_3()

In [ ]:
A, B, C = split_channels_3(X_train)
callbacks = create_callbacks('model10.h5')
history = model10.fit([A,B,C], y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C = split_channels_3(X_test)
test_results(model10, [A,B,C], y_test)

# Model 11

In [ ]:
def split_channels_4(X):
    A = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z'], axis=1)
    B = X.drop(['accelometer_x','accelometer_y','accelometer_z'], axis=1)
    C = X.drop(['gyroscope_y','gyroscope_z','accelometer_y','accelometer_z'], axis=1)
    D = X.drop(['gyroscope_x','gyroscope_z','accelometer_x','accelometer_z'], axis=1)
    return A, B, C, D

def create_model_channel_4():
    inp1, c1 = create_chanel(deapth=2,N=64)
    inp2, c2 = create_chanel(N=64)
    inp3, c3 = create_chanel(size=2,deapth=2,N=12)
    inp4, c4 = create_chanel(size=2,deapth=2,N=12)
    x = concatenate([c1, c2, c3, c4])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2, inp3, inp4], outputs=x, name='model11') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model11 = create_model_channel_4()

In [ ]:
A, B, C, D = split_channels_4(X_train)
callbacks = create_callbacks('model11.h5')
history = model11.fit([A, B, C, D], y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C, D = split_channels_4(X_test)

test_results(model11, [A, B, C, D], y_test)

# Model 13

In [ ]:
def go_depper(inputs, deapth=1):
    x = concatenate(inputs)
    a ,N = x.shape 
    if deapth!=0:
        x = dense_deapth_block(x, deapth=deapth, N=N)
    x = Dense(8)(x)
    return x
    
def go_all_deper(inputs, group_size=2, deapth=2):
    size = len(inputs)
    xs = []
    for i in range(size):
        if size - i - group_size >= 0:
            group = inputs[i:i+group_size]
            x = go_depper(group, deapth=deapth)
            xs.append(x)
    if len(xs) == 1:
        return xs[0]
    x = concatenate(xs)
    return x

In [ ]:
def create_model_channel_4_2():
    inp1, c1 = create_chanel(deapth=2,N=32)
    inp2, c2 = create_chanel(N=32)
    inp3, c3 = create_chanel(size=2,deapth=2,N=6)
    inp4, c4 = create_chanel(size=2,deapth=2,N=6)
    
    inp5 = concatenate([inp1,inp2])
    c5 = dense_deapth_block(inp5,deapth=2,N=32)
    
    inp6 = concatenate([inp3,inp4])
    c6 = dense_deapth_block(inp6,deapth=2,N=32)
    
    x = concatenate([c1,c2, c3, c4, c5, c6])
    x = Dense(8)(x)
    x = Activation('softmax')(x)
    model = Model(inputs=[inp1, inp2, inp3, inp4], outputs=x, name='model13') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model13 = create_model_channel_4_2()
model13.summary()

In [ ]:
A, B, C, D = split_channels_4(X_train)
callbacks = create_callbacks('model13.h5')
history = model13.fit([A, B, C, D], y_train, epochs=EPOCKS, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C, D = split_channels_4(X_test)
test_results(model13, [A, B, C, D], y_test)

# Model 12

In [ ]:
def split_channels_5(X):
    A = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z'], axis=1)
    B = X.drop(['accelometer_x','accelometer_y','accelometer_z'], axis=1)
    C = X.drop(['gyroscope_y','gyroscope_z','accelometer_y','accelometer_z'], axis=1)
    D = X.drop(['gyroscope_x','gyroscope_z','accelometer_x','accelometer_z'], axis=1)
    E = X.drop(['gyroscope_x','gyroscope_y','accelometer_x','accelometer_y'], axis=1)
    return A, B, C, D, E

def create_model_channel_5():
    inp1, c1 = create_chanel(deapth=2,N=32)
    inp2, c2 = create_chanel(N=32)
    inp3, c3 = create_chanel(size=2,deapth=2,N=8)
    inp4, c4 = create_chanel(size=2,deapth=2,N=8)
    inp5, c5 = create_chanel(size=2,deapth=2,N=8)
    
    
    inp6= concatenate([inp1,inp2])
    c6 = dense_deapth_block(inp6,deapth=2,N=32,extend_N=16)
    
    inp7 = concatenate([inp3,inp4])
    c7 = dense_deapth_block(inp7,deapth=2,N=8)
    
    inp8 = concatenate([inp4,inp5])
    c8 = dense_deapth_block(inp8,deapth=2,N=8)
    
    inp9 = concatenate([inp3,inp5])
    c9 = dense_deapth_block(inp9,deapth=2,N=8)
    
    x = concatenate([c1,c2, c3, c4, c5, c6, c7, c8, c9])
    x = Dense(100)(x)
    x = PReLU()(x)
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2, inp3, inp4, inp5], outputs=x, name='model12') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model12 = create_model_channel_5()

In [ ]:
A, B, C, D, E = split_channels_5(X_train)
callbacks = create_callbacks('model12.h5')
history = model12.fit([A, B, C, D, E], y_train, epochs=200, batch_size=64, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C, D, E = split_channels_5(X_test)
test_results(model12, [A, B, C, D, E], y_test)

# Model 14

In [ ]:
def create_model_channel_5_2():
    inp1, c1 = create_chanel(deapth=2,N=32)
    inp2, c2 = create_chanel(N=32)
    inp3, c3 = create_chanel(size=2,deapth=2,N=8)
    inp4, c4 = create_chanel(size=2,deapth=2,N=8)
    inp5, c5 = create_chanel(size=2,deapth=2,N=8)
    
    inp7 = concatenate([inp3,inp4])
    c7 = dense_deapth_block(inp7,deapth=2,N=8)
    
    inp8 = concatenate([inp4,inp5])
    c8 = dense_deapth_block(inp8,deapth=2,N=8)
    
    inp9 = concatenate([inp3,inp5])
    c9 = dense_deapth_block(inp9,deapth=2,N=8)
    
    x2 = concatenate([c1, c2, c3, c4, c5, c7, c8, c9])
    x2 = Dense(8)(x2)
    x2 = PReLU()(x2)
    
    x1 = concatenate([c1, c2, c3, c4, c5])
    x1 = Dense(8)(x1)
    x1 = PReLU()(x1)
    
    x0 = concatenate([c1, c2])
    x0 = Dense(8)(x0)
    x0 = PReLU()(x0)
    
    x = concatenate([x0,x1,x2])
    x = Dense(8, activation='softmax')(x)
    
    model = Model(inputs=[inp1, inp2, inp3, inp4, inp5], outputs=x, name='model14') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model14 = create_model_channel_5_2()
model14.summary()

In [ ]:
A, B, C, D, E = split_channels_5(X_train)
callbacks = create_callbacks('model14.h5')
history = model14.fit([A, B, C, D, E], y_train, epochs=200, batch_size=64, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C, D, E = split_channels_5(X_test)
test_results(model14, [A, B, C, D, E], y_test)

# Model 15

In [ ]:
print(X_train_d.columns)

In [ ]:
def split_channels_4_d(X):
    A = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z', 
                'sub_gyroscope_x','sub_gyroscope_y','sub_gyroscope_z', 
                'sub_accelometer_x','sub_accelometer_y','sub_accelometer_z'], axis=1)
    B = X.drop(['accelometer_x','accelometer_y','accelometer_z',
                'sub_gyroscope_x','sub_gyroscope_y','sub_gyroscope_z', 
                'sub_accelometer_x','sub_accelometer_y','sub_accelometer_z'], axis=1)
    C = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z', 
                'sub_gyroscope_x','sub_gyroscope_y','sub_gyroscope_z', 
                'accelometer_x','accelometer_y','accelometer_z'], axis=1)
    D = X.drop(['gyroscope_x','gyroscope_y','gyroscope_z', 
                'accelometer_x','accelometer_y','accelometer_z', 
                'sub_accelometer_x','sub_accelometer_y','sub_accelometer_z'], axis=1)
    return A, B, C, D

def create_model_channel_4_d():
    inp1, c1 = create_chanel(deapth=2,N=32)
    inp2, c2 = create_chanel(N=32)
    inp3, c3 = create_chanel(N=32)
    inp4, c4 = create_chanel(N=32)
    x = concatenate([c1, c2, c3, c4])
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=[inp1, inp2, inp3, inp4], outputs=x, name='model15') 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=create_metrics())
    return model

model15 = create_model_channel_4_d()
model15.summary()

In [ ]:
A, B, C, D = split_channels_4_d(X_train_d)
callbacks = create_callbacks('model15.h5')
history = model15.fit([A, B, C, D], y_train_d, epochs=200, batch_size=64, validation_split=0.2, callbacks=callbacks)

In [ ]:
quick_plot_history(history)
A, B, C, D = split_channels_4_d(X_test_d)
test_results(model15, [A, B, C, D], y_test_d)

# Model 16

In [ ]:
X_train_d_3, X_test_d_3, y_train_d_3, y_test_d_3 = split_train_test_by_sliding_window(X, y, N=3)
y_train_d_3, y_test_d_3, y_dic_d_3 = y_to_categorical(y_train_d_3, y_test_d_3)

In [ ]:
print(X_train_d_3.columns)

# Regression Problem

In [ ]:
def tag_to_vector(tag):
    vx = 5
    vy = 5
    if tag == 'up': 
        return [0, vy]
    elif tag == 'down':
        return [0, -vy]
    elif tag == 'right':
        return [vx, 0]
    elif tag == 'left':
        return [-vx, 0]
    elif tag == 'upright':
        return [vx, vy]
    elif tag == 'downleft':
        return [-vx, -vy]
    elif tag == 'upleft':
        return [-vx, vy]
    #elif tag =='downright'
    return [vx, -vy]

def y_to_vector_speed(y_train, y_test):
    y_train = [tag_to_vector(x) for x in y_train]
    y_test = [tag_to_vector(x) for x in y_test]
    y_train = y_train.to_numpy()
    print('Changed to vector','y train shape: ',len(y_train), ' y test shape: ', len(y_test))
    return y_train, y_test
    
X_train, X_test, y_train, y_test = split_train_test(X, y)
y_train, y_test = y_to_vector_speed(y_train, y_test)